In [70]:
import wrds
import pandas as pd
import numpy as np
from datetime import datetime

In [71]:
def wrds_connection():
    """Create a WRDS database connection."""
    db = wrds.Connection()
    return db

###############################################################################
#                             CRSP QUERIES                                    #
###############################################################################

def get_crsp_monthly_data(db, start_date='2000-01-01', end_date='2005-12-31'):
    """
    Pull monthly CRSP data (msf) within [start_date, end_date].
    We include permno, date (month-end), ret, prc, shrout, and compute market cap.
    """
    query = f"""
    SELECT m.permno,
       m.date,
       m.ret,
       m.prc,
       m.shrout,
       m.vol,
       m.retx
    FROM crsp.msf m
    JOIN crsp.stocknames n
    ON m.permno = n.permno
     AND m.date BETWEEN n.namedt AND n.nameenddt
    WHERE m.date BETWEEN '{start_date}' AND '{end_date}'
     AND n.shrcd IN (10, 11);
    """
    df = db.raw_sql(query, date_cols=['date'])
    
    # Clean up
    df.dropna(subset=['permno','date','prc','shrout'], inplace=True)
    df['prc'] = df['prc'].abs()  # CRSP can store negative prices
    df['ret'] = pd.to_numeric(df['ret'], errors='coerce')
    df['shrout'] = pd.to_numeric(df['shrout'], errors='coerce')
    
    # Market cap in thousands of dollars (prc * shrout).
    # For ranking, the scale doesn’t matter, but you could multiply by 1,000 if you need exact $.
    df['mcap'] = df['prc'] * df['shrout']
    
    return df

def filter_top_n_by_year(df_crsp, N=50):
    """
    Process the dataframe in the following steps:
      1. Extract the year from the date.
      2. For each year and company (permno), compute the maximum market cap.
      3. For each year, identify the top N companies and record (if not already recorded) 
         the year of their first appearance in the top N.
      4. Map this "entry_year" back onto the original dataframe.
      5. Filter the dataframe to include only companies that have ever been in the top N,
         but retain all of their data (even data from before their entry year).
         
    The resulting dataframe contains an extra column, 'entry_year', indicating the first year 
    the company entered the top N.
    
    Parameters:
      df_crsp (pd.DataFrame): DataFrame containing at least:
                              - 'date': dates (datetime or convertible)
                              - 'mcap': market capitalization
                              - 'permno': unique company identifier.
      N (int): Number of top companies to select per year (default 50).
      
    Returns:
      pd.DataFrame: Updated DataFrame that includes all data for companies that have ever been in 
                    the top N, along with a new column 'entry_year' for each company.
    """
    # Step 1: Ensure date is in datetime format and extract the year.
    if not pd.api.types.is_datetime64_any_dtype(df_crsp['date']):
        df_crsp['date'] = pd.to_datetime(df_crsp['date'])
    df_crsp['year'] = df_crsp['date'].dt.year

    # Step 2: For each year and permno, compute the maximum market cap for that year.
    df_yearly = df_crsp.groupby(['year', 'permno'], as_index=False)['mcap'].max()
    
    # Initialize a dictionary to record each permno's first entry year into the top N.
    entry_year_dict = {}
    
    # Process the years in ascending order.
    years = sorted(df_yearly['year'].unique())
    for yr in years:
        # Select data for the current year.
        df_current = df_yearly[df_yearly['year'] == yr]
        # Sort companies by mcap (descending) and take the top N.
        df_current_sorted = df_current.sort_values('mcap', ascending=False)
        top_permnos = df_current_sorted.head(N)['permno'].unique()
        
        # For each company in the top N, if we haven’t yet recorded an entry year, record it.
        for permno in top_permnos:
            if permno not in entry_year_dict:
                entry_year_dict[permno] = yr
    
    # Step 4: Map the entry year back to the original dataframe.
    df_crsp['entry_year'] = df_crsp['permno'].map(entry_year_dict)
    
    # Step 5: Filter out companies that never made the top N.
    df_filtered = df_crsp[df_crsp['entry_year'].notnull()].copy()
    
    # Optionally drop the temporary 'year' column.
    df_filtered.drop(columns=['year'], inplace=True)
    
    return df_filtered

###############################################################################
#                       COMPUSTAT (Quarterly) + LINKING                       #
###############################################################################

def get_compustat_quarterly_fundamentals(db, start_date='2000-01-01', end_date='2005-12-31'):
    """
    Pull quarterly fundamentals from comp.fundq, restricting to [start_date, end_date]
    based on datadate (the quarter-end date).
    
    We'll define a 'release_date' using rdq if available, else datadate + 45 days.
    """
    query = f"""
        SELECT gvkey,
               datadate,    -- Quarter-end date
               rdq,         -- Earnings announcement date
               fyearq, fqtr,
               atq,         -- Total assets
               actq,        -- Current Assets
               ceqq,        -- Common/Ordinary Equity - Total
               cheq,        -- Cash and Cash Equivs
               cogsq,       -- Cost of Goods Sold
               dlcq,        -- Debt in Current Liabilities
               dlttq,       -- Long-Term Debt - Total
               dpq,         -- Depreciation and Amortization - Total
               ibq,         -- Income Before Extraordinary Item
               invtq,       -- Inventories - Total
               lctq,        -- Current Liabilities
               ltq,         -- Liabilities Total
               niq,         -- Net income
               ppentq,      -- Property Plant and Equipment - Total
               saleq,       -- Sales/Turnover (Net)
               seqq         -- Stockholders Equity
               
        FROM comp.fundq
        WHERE indfmt='INDL'
          AND datafmt='STD'
          AND popsrc='D'
          AND consol='C'
          AND datadate BETWEEN '{start_date}' AND '{end_date}'
    """
    df_q = db.raw_sql(query, date_cols=['datadate','rdq'])
    
    # Approximate the release_date
    df_q['release_date'] = df_q['rdq']
    missing_rdq = df_q['release_date'].isna()
    # If rdq is missing, assume 45 days after quarter-end
    df_q.loc[missing_rdq, 'release_date'] = df_q.loc[missing_rdq, 'datadate'] + pd.Timedelta(days=45)
    
    df_q.dropna(subset=['gvkey','datadate','release_date'], inplace=True)
    
    return df_q

def link_ccm(db, df_fundq):
    """
    Link quarterly fundamentals (gvkey) to CRSP (permno) via the CCM link table,
    keeping only valid link types and ensuring datadate is within link date range.
    """
    ccm_query = """
        SELECT gvkey,
               lpermno AS permno,
               linkdt,
               linkenddt
        FROM crsp.ccmxpf_linktable
        WHERE linktype IN ('LC','LU','LX','LD','LS','LN')
          AND linkprim IN ('P','C','J')
    """
    df_link = db.raw_sql(ccm_query, date_cols=['linkdt','linkenddt'])
    df_link['linkenddt'] = df_link['linkenddt'].fillna(pd.to_datetime('2099-12-31'))
    
    # Merge fundamentals with link table
    df_merged = pd.merge(df_fundq, df_link, on='gvkey', how='left')
    
    # Keep rows where datadate is within link date range
    mask = (df_merged['datadate'] >= df_merged['linkdt']) & \
           (df_merged['datadate'] <= df_merged['linkenddt'])
    df_merged = df_merged[mask].copy()
    df_merged['permno'] = df_merged['permno'].astype('int64')
    
    return df_merged

###############################################################################
#                    CARRY-FORWARD MERGE (MONTHLY CRSP)                       #
###############################################################################

def carry_forward_monthly_fundamentals(df_crsp_m, df_ccm_q):
    """
    Attach fundamentals to monthly CRSP by carrying forward the 
    most recent statement (release_date <= CRSP date).
    
    Steps:
      1. Sort CRSP data by (permno, date).
      2. Sort fundamentals by (permno, release_date).
      3. merge_asof(direction='backward', by='permno')
      4. Drop rows with no matched fundamentals (if desired).
    """
    df_crsp_m.sort_values(by=['date','permno'], ascending=[True, True], inplace=True)
    df_ccm_q.sort_values(by=['release_date','permno'], ascending=[True, True], inplace=True)
    
    merged = pd.merge_asof(
        left=df_crsp_m,
        right=df_ccm_q,
        left_on='date',
        right_on='release_date',
        by='permno',
        direction='backward'
    )
    
    # Remove rows with no fundamentals (no prior statement)
    merged.dropna(subset=['gvkey'], inplace=True)
    
    return merged

###############################################################################
#                              CLEAN DATA                                     #
###############################################################################

def filter_data(df, n):
    """
    Cleans a DataFrame that contains monthly data for companies (identified by 'permno')
    in two steps:
    
    1. Month Filtering:
       Only keep months (year-month periods) where the number of unique companies exceeds n.
    
    2. Recursive Month-to-Month Validation (Within Each Year):
       For each year, start with the companies in the first month.
       Then for each subsequent month, only include a company's data if that company was present in
       the previous month. This check is applied month-by-month until reaching a month where there is
       no earlier month in that year (the first month is always accepted).
       
       In effect, a company’s data for a given month is included only if it has a continuous presence 
       (from some start month of that year) up to that month.
    
    Parameters:
      df (pd.DataFrame): DataFrame containing at least:
                         - 'date': monthly dates (or strings convertible to datetime)
                         - 'permno': unique company identifier.
      n (int): Minimum number of unique companies that must be present in a month for it to be kept.
    
    Returns:
      pd.DataFrame: The filtered DataFrame.
    """
    # --- Preparation: Ensure dates and add helper columns ---
    df = df.copy()
    if not pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = pd.to_datetime(df['date'])
    
    # Create a 'year' column and a 'year_month' Period column (e.g., "2020-01").
    df['year'] = df['date'].dt.year
    df['year_month'] = df['date'].dt.to_period('M')
    
    # --- Step 1: Month Filtering ---
    # Count unique companies per month.
    month_counts = df.groupby('year_month')['permno'].nunique()
    # Identify valid months where the unique count is greater than n.
    valid_months = month_counts[month_counts >= n/(1.25)].index
    df_filtered = df[df['year_month'].isin(valid_months)].copy()
    
    # --- Step 2: Recursive Month-to-Month Validation ---
    # We will process each year separately.
    cleaned_years = []  # list to hold cleaned data for each year.
    
    for yr, group in df_filtered.groupby('year'):
        # Sort the group by date to ensure months are in order.
        group = group.sort_values('date').copy()
        # Get the unique months for the year in order.
        months = sorted(group['year_month'].unique())
        
        # This dictionary will map each month to the set of companies that "pass" the chain for that month.
        valid_by_month = {}
        
        for i, m in enumerate(months):
            # Get the set of companies in month m.
            companies_current = set(group[group['year_month'] == m]['permno'].unique())
            
            if i == 0:
                # First month of the year: accept all companies.
                valid_by_month[m] = companies_current
            else:
                # For subsequent months, only keep companies that were valid in the immediately previous month.
                prev_month = months[i-1]
                valid_by_month[m] = companies_current.intersection(valid_by_month[prev_month])
        
        # Now, for each month in this year, filter rows to only those companies that passed the chain.
        frames = []
        for m in months:
            valid_companies = valid_by_month[m]
            # Filter the data for month m to only include rows for valid companies.
            frames.append(group[(group['year_month'] == m) & (group['permno'].isin(valid_companies))])
        
        if frames:
            cleaned_years.append(pd.concat(frames))
    
    # Combine all years back together.
    df_cleaned = pd.concat(cleaned_years, ignore_index=True)
    
    # Optionally, drop helper columns.
    df_cleaned.drop(columns=['year', 'year_month'], inplace=True)
    
    return df_cleaned

def impute_fundamentals(df):
    """Cleans & imputes missing values for key financial variables"""
    
    # Ensure the DataFrame is sorted by company and date
    df.sort_values(by=["permno", "date"], inplace=True)
    
    # Convert 0s to NaN for selected variables (verify that 0 is not a valid value here)
    zero_to_nan_cols = ["dpq", "saleq", "cogsq"]
    df[zero_to_nan_cols] = df[zero_to_nan_cols].replace(0, np.nan)
    
    # Forward and backward fill for balance sheet items (these items tend to change slowly)
    balance_sheet_cols = ["atq", "actq", "ceqq", "cheq", "cogsq", "dlcq", "dlttq", "dpq", "invtq", "lctq", "ltq", "ppentq", "seqq"]
    df[balance_sheet_cols] = df.groupby("permno")[balance_sheet_cols].fillna(method="ffill").fillna(method="bfill")
    
    # Rolling mean imputation for trend-based variables (e.g., earnings measures)
    trend_cols = ["ibq", "niq", "saleq"]
    for col in trend_cols:
        df[col] = df.groupby("permno")[col].transform(lambda x: x.fillna(x.rolling(4, min_periods=1).mean()))
    
    return df

def clean_data(df):
    """
    Cleans the input DataFrame by:
      1. Handling missing values using forward-fill, backward-fill, and rolling mean.
      2. Converting date columns to datetime format.
      3. Removing duplicate rows.
      4. Fixing data types for categorical and numerical columns.
    """
    
    # 1. Handle Missing Values & Impute Fundamentals
    df = impute_fundamentals(df)
    
    # 2. Drop unnecessary columns
    drop_cols = ["gvkey", "datadate", "rdq", "fyearq", "fqtr", "release_date", "linkdt", "linkenddt"]
    for col in drop_cols:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
    
    # 2. Convert Date Columns to Datetime
    df["date"] = pd.to_datetime(df["date"])
    
    # 3. Remove Duplicate Entries
    df.drop_duplicates(inplace=True)
    
    return df

def rename_columns(df):
    """
    Rename columns to more descriptive names.
    """
    new_columns = {
        "atq": "total_assets",
        "actq": "current_assets",
        "ceqq": "common_equity",
        "cheq": "cash_equivalents",
        "cogsq": "cost_of_goods_sold",
        "dlcq": "debt_current",
        "dlttq": "debt_long_term",
        "dpq": "depreciation_amortization",
        "ibq": "income_before_extraordinary_items",
        "invtq": "inventories",
        "lctq": "liabilities_current",
        "ltq": "liabilities_total",
        "mcap": "market_cap",
        "niq": "net_income",
        "ppentq": "property_plant_equipment",
        "prc": "price",
        "ret": "return",
        "retx": "return_ex_dividends",
        "saleq": "sales",
        "seqq": "stockholders_equity",
        "shrout": "shares_outstanding",
        "vol": "volume"
    }
    
    df.rename(columns=new_columns, inplace=True)
    
    return df



In [72]:
# Define a single date range for BOTH CRSP and Compustat
START_DATE = '2010-01-01'
END_DATE   = '2020-12-31'
TOP_N      = 50

db = wrds_connection()

# 1. Pull monthly CRSP in [start_date, end_date]
crsp_m = get_crsp_monthly_data(db, start_date=START_DATE, end_date=END_DATE)
print("Initial CRSP monthly shape:", crsp_m.shape)

# 2. Filter to top N by market cap, across the entire timeframe
crsp_m_topN = filter_top_n_by_year(crsp_m, N=TOP_N)
print(f"CRSP after filtering to top {TOP_N} by market cap, shape:", crsp_m_topN.shape)

# 3. Pull quarterly fundamentals from Compustat, restricted to same [start_date, end_date]
comp_q = get_compustat_quarterly_fundamentals(db, start_date=START_DATE, end_date=END_DATE)
print("Quarterly fundamentals shape:", comp_q.shape)

# 4. Link to CRSP via CCM
comp_q_ccm = link_ccm(db, comp_q)
print("After CCM linking shape:", comp_q_ccm.shape)

# 5. Carry-forward fundamentals for monthly CRSP
merged_df = carry_forward_monthly_fundamentals(crsp_m_topN, comp_q_ccm)
print("Merged shape:", merged_df.shape)

# 6. Filter data to remove odd companies who do not have data for the entire year
filtered_df = filter_data(merged_df, n=TOP_N)
print("Filtered shape:", filtered_df.shape)

# 7. Clean data to handle missing values, etc.
cleansed_df = clean_data(filtered_df)
final_df = rename_columns(cleansed_df)
print("Final shape:", final_df.shape)

# Example look at a few columns
final_df.head(10)

# Save result
final_df.to_csv("monthly_top50_same_range_carried_forward.csv", index=False)
print("\nData pipeline complete! Saved to monthly_top50_same_range_carried_forward.csv")

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Initial CRSP monthly shape: (489969, 8)
CRSP after filtering to top 50 by market cap, shape: (10548, 9)
Quarterly fundamentals shape: (507282, 22)
After CCM linking shape: (308636, 25)
Merged shape: (10310, 33)
Filtered shape: (10149, 33)
Final shape: (10149, 25)

Data pipeline complete! Saved to monthly_top50_same_range_carried_forward.csv


/var/folders/y0/flcpjk4s0bqdtnhxz0py83c80000gn/T/ipykernel_1185/3155818210.py:318: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use DataFrame.fillna instead
  df[balance_sheet_cols] = df.groupby("permno")[balance_sheet_cols].fillna(method="ffill").fillna(method="bfill")
/var/folders/y0/flcpjk4s0bqdtnhxz0py83c80000gn/T/ipykernel_1185/3155818210.py:318: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[balance_sheet_cols] = df.groupby("permno")[balance_sheet_cols].fillna(method="ffill").fillna(method="bfill")
